In [12]:
import os
import ast
import logging
import pprint
import json
import time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.column import *
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import pprint
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

pp = pprint.PrettyPrinter(indent = 1)
print("imported modules.")

imported modules.


## creds.json

In [13]:
with open("./../creds.json", "r") as g:
    creds = json.load(g)
    g.close()

print("creds.json keys: ")
pp.pprint([k for k in creds.keys()])

creds.json keys: 
['twitter-api-key',
 'twitter-secret-key',
 'twitter-bearer',
 'twitter-access-token',
 'twitter-secret-access',
 'aws-access-key',
 'aws-secret-access-key',
 'spark_host',
 'spark_port',
 'dynamo_host',
 'dynamo_port',
 'extra_jar_list']


## boto3 clients

In [14]:
s3 = boto3.client("s3")
athena = boto3.client("athena")
glue = boto3.client("glue")
my_lambda = boto3.client("lambda")
dynamo = boto3.client("dynamodb")
print("connected to boto3 clients.")

connected to boto3 clients.


## delta table s3 path

In [15]:
bucket = "reddit-streaming-stevenhurwitt-new"
subreddit = "aws"
filepath = os.path.join("s3a://", bucket, subreddit + "_clean")
print(filepath)

s3a://reddit-streaming-stevenhurwitt-new/aws_clean


## spark

In [17]:
spark_host = creds["spark_host"]
spark_port = creds["spark_port"]
aws_client = creds["aws-access-key"]
aws_secret = creds["aws-secret-access-key"]
extra_jar_list = creds["extra_jar_list"]

In [18]:
spark = SparkSession.builder.appName("twitter") \
    .master("spark://{}:{}".format(spark_host, spark_port)) \
    .config("spark.executor.memory", "2048m") \
    .config("spark.executor.cores", "2") \
    .config("spark.streaming.concurrentJobs", "8") \
    .config("spark.local.dir", "/opt/workspace/tmp/driver/") \
    .config("spark.worker.dir", "/opt/workspace/tmp/executor/") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "/opt/workspace/tmp/events/") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.jars.packages", extra_jar_list) \
    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.buffer.dir", "/opt/workspace/tmp/blocks") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext
index = 0
sc.setLogLevel("WARN")
sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
print("imported modules, created spark.")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/steven/.ivy2/cache
The jars for the packages stored in: /home/steven/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8fc365f1-c5e8-4f4c-90ab-557c0eb58f1b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in local-m2-cache
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.32 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in local-m2-cache
	fo

RuntimeError: Java gateway process exited before sending its port number

In [19]:
df = spark.read.format("delta").option("header", True).load(filepath)
df.show()

NameError: name 'spark' is not defined